In [ ]:
import numpy as np
import MDAnalysis as mda

from MDRestraintsGenerator import search as _search
from MDRestraintsGenerator.restraints import FindBoreschRestraint as _FindBoreschRestraint

In [ ]:
u = mda.Universe('topology.prmtop', 'output.dcd')
u

In [ ]:
list(u.residues)

In [ ]:
# search limited to backbone atoms
receptor_selection_str = 'resid 35-68 and name CA C N' # residue indices need to be substituted for each system
ligand_selection_str = 'resid 1-34 and name CA C N' 

In [ ]:
ligand_atoms = _search.find_ligand_atoms(u, l_selection=ligand_selection_str, p_align=receptor_selection_str)

In [ ]:
atom_set = []
for l_atoms in ligand_atoms:
    psearch = _search.FindHostAtoms(u, l_atoms[0], p_selection=receptor_selection_str)
    psearch.run()
    atom_set.extend([(l_atoms, p) for p in psearch.host_atoms])

In [ ]:
boresch = _FindBoreschRestraint(u, atom_set)
boresch.run()

In [ ]:
# Index gets reassigned multiple times
# https://github.com/IAlibay/MDRestraintsGenerator/blob/master/MDRestraintsGenerator/datatypes.py#L810-L825
l1_idx, r1_idx = boresch.restraint.bond.atomgroup.atoms.ix
l2_idx, l1_idx, r1_idx = boresch.restraint.angles[0].atomgroup.atoms.ix
l1_idx, r1_idx, r2_idx = boresch.restraint.angles[1].atomgroup.atoms.ix
l3_idx, l2_idx, l1_idx, r1_idx = boresch.restraint.dihedrals[0].atomgroup.atoms.ix
l2_idx, l1_idx, r1_idx, r2_idx = boresch.restraint.dihedrals[1].atomgroup.atoms.ix
l1_idx, r1_idx, r2_idx, r3_idx = boresch.restraint.dihedrals[2].atomgroup.atoms.ix

In [ ]:
stable_residues = []
stable_points = [l1_idx, l2_idx, l3_idx, r1_idx, r2_idx, r3_idx]

for point in stable_points:
    atom = u.atoms[point]
    residue = atom.residue
    stable_residues.append(residue)

In [ ]:
stable_residues = list(set(stable_residues))
stable_residues